# Iris Classification Tutorial

Tutorial by the AI Guy walking through solving the Iris Dataset with Scikit Learn.

## 1. Import Required Dependencies

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn import datasets

## 2. Load Iris Dataset

In [3]:
iris = datasets.load_iris()
type(iris)

sklearn.utils.Bunch

## 3. Summarize your Dataset

In [4]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
# print type for the data and labels
type(iris.data), type(iris.target)

(numpy.ndarray, numpy.ndarray)

In [6]:
iris.data.shape

(150, 4)

In [7]:
# print labels
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

## 4. Peek your data

In [8]:
# perform EDA
X = iris.data
Y = iris.target
df = pd.DataFrame(X, columns = iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


## 4.1 Statistical Summary

In [9]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


## 4.2 Class Distribution

In [10]:
print(df.groupby(iris.target).size())

0    50
1    50
2    50
dtype: int64


## 5. Data Visualization

## 6. Evaluate your Algorithms

## 7. Make Predictions